In [3]:
import torch
use_cuda = torch.cuda.is_available()

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

__CUDNN VERSION: 8500
__Number CUDA Devices: 1
__CUDA Device Name: Tesla T4
__CUDA Device Total Memory [GB]: 15.843721216


In [ ]:
# !pip install torch
# !pip install pandas_datareader
# !pip install matplotlib
# !pip install seaborn
# !pip install scipy
# !pip install 
# !pip install scikit-learn
# !pip install xgboost

In [1]:
1+1

2

In [8]:
from pandas_datareader import data as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

from   scipy.io import loadmat
import time
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# from utility_funcs import *

import warnings
warnings.filterwarnings('ignore')

In [8]:
# factors = pd.read_parquet('/Users/shaorui/Desktop/CMA/uqer_ALL_2018_20211029.parquet')
citi_path = '/home/ubuntu/multi_factor_model/raw_data/citi_sector_33_info.csv'
Andariel_path = '/home/ubuntu/multi_factor_model/raw_data/_Andariel_20230426.mat'
DIABLO_FILE = '/Users/shaorui/Desktop/CMA/Diablo_EVA_20230411.parquet'

data_mat    = loadmat(Andariel_path)

In [9]:
def read_Andariel(Andariel_path, citi_path):
    
    ############################# BASIC-info <<<>>> JuYuan by Matlab 
    data_mat    = loadmat(Andariel_path)
    citi_sector = pd.read_csv(citi_path, encoding="gbk")
    jstockcode  = data_mat['Data'][0,0]['jstockcode'].ravel()
    jipoday     = data_mat['Data'][0,0]['jipodate'].ravel()
    jquitday    = data_mat['Data'][0,0]['jquitday'].ravel()
    jstockname  = data_mat['Data'][0,0]['jstockname'].ravel()
    EOM         = np.array(data_mat['Data'][0,0]['date_EOM']).ravel()
    EOW         = np.array(data_mat['Data'][0,0]['date_EOW']).ravel()
    ALLDAYS     = np.array(data_mat['Data'][0,0]['ALLDAY']).ravel()
    ST          = np.asarray(data_mat['Data'][0,0]['ST'])
    Citics1     = np.asarray(data_mat['Data'][0,0]['Citics'])[:,-1]
    CITI        = np.zeros_like(Citics1)
    for i in range(len(CITI)):  #slow
        CITI[i] = citi_sector.loc[citi_sector['code']==Citics1[i]]['id']
    CITI[np.where(CITI==31)]=30
    windforce = pd.DataFrame({'code':jstockcode,'name':jstockname,'ipoday':jipoday,'quitday':jquitday,'citics_sector':Citics1,'sector':CITI})
    
    ZZ500info      = data_mat['GUNDAM'][0,0]['ZZ500info'] 
    ZZ500_DAYS     = [int(i) for i in ZZ500info[0,:]['date']]
    INDEXPRICE_500 = data_mat['GUNDAM'][0,0]['Index_Price_ZZ500']  # day close open
    
    # HS300info  = data_mat['GUNDAM'][0,0]['HS300info'] 
    # HS300_DAYS = [int(i) for i in HS300info[0,:]['date']]
    # INDEXPRICE_300 = data_mat['GUNDAM'][0,0]['Index_Price_HS300']  # day close open

    andariel = {'jstockcode':jstockcode,
                'jipoday':jipoday,
                'jipoday':jipoday,
                'jquitday':jquitday,
                'jstockname':jstockname,
                'EOM':EOM,
                'EOW':EOW,
                'ALLDAYS':ALLDAYS,
                'ST':ST,
                'citi_sector':citi_sector,
                'windforce':windforce,
                'ZZ500info':ZZ500info,
                'ZZ500_DAYS':ZZ500_DAYS,
                'INDEXPRICE_500':INDEXPRICE_500
                }
    return andariel

andariel = read_Andariel(Andariel_path, citi_path)

In [51]:
andariel['ZZ500info'][0,-1]

(array([[20230331]], dtype=int32), array([[9.0000000e+00, 2.0230331e+07, 3.4100000e-03],
       [1.2000000e+01, 2.0230331e+07, 1.3400000e-03],
       [2.1000000e+01, 2.0230331e+07, 2.2800000e-03],
       ...,
       [6.8877700e+05, 2.0230331e+07, 5.2500000e-03],
       [6.8877900e+05, 2.0230331e+07, 1.6800000e-03],
       [6.8881900e+05, 2.0230331e+07, 6.8000000e-04]]))

In [39]:
andariel['INDEXPRICE_500'][-1]

array([2.0230426e+07, 6.1421541e+03, 6.0951335e+03])

In [52]:
andariel['ZZ500_DAYS']

[20100129,
 20100226,
 20100331,
 20100430,
 20100531,
 20100630,
 20100730,
 20100831,
 20100930,
 20101029,
 20101130,
 20101231,
 20110131,
 20110228,
 20110331,
 20110429,
 20110531,
 20110630,
 20110729,
 20110831,
 20110930,
 20111031,
 20111130,
 20111230,
 20120131,
 20120229,
 20120330,
 20120427,
 20120531,
 20120629,
 20120731,
 20120831,
 20120928,
 20121031,
 20121130,
 20121231,
 20130131,
 20130228,
 20130329,
 20130426,
 20130531,
 20130628,
 20130731,
 20130830,
 20130930,
 20131031,
 20131129,
 20131231,
 20140130,
 20140228,
 20140331,
 20140430,
 20140530,
 20140630,
 20140731,
 20140829,
 20140930,
 20141031,
 20141128,
 20141231,
 20150130,
 20150227,
 20150331,
 20150430,
 20150529,
 20150630,
 20150731,
 20150831,
 20150930,
 20151030,
 20151130,
 20151231,
 20160129,
 20160229,
 20160331,
 20160429,
 20160531,
 20160630,
 20160729,
 20160831,
 20160930,
 20161031,
 20161130,
 20161230,
 20170126,
 20170228,
 20170331,
 20170428,
 20170531,
 20170630,
 20170731,